In [1]:
import sqlite3

In [2]:
DATAPATH = 'football.db'

In [3]:
def setup_database(datapath):
    bdd = sqlite3.connect(datapath)
    cur = bdd.cursor()

    # Supprimer les tables existantes si elles existent
    cur.execute("DROP TABLE IF EXISTS matchs")
    cur.execute("DROP TABLE IF EXISTS equipes")

    # Créer la table 'equipes' en premier
    cur.execute(''' 
    CREATE TABLE IF NOT EXISTS equipes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nom VARCHAR(30),
        federation VARCHAR(30),
        confederation VARCHAR(30)
    ) ''')

    # Créer la table 'matchs' avec les clés étrangères
    cur.execute(''' 
    CREATE TABLE IF NOT EXISTS matchs (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date DATE,
        domicile INTEGER,
        exterieur INTEGER,
        score_dom INTEGER,
        score_ext INTEGER,
        competition VARCHAR(30),
        FOREIGN KEY (domicile) REFERENCES equipes(id),
        FOREIGN KEY (exterieur) REFERENCES equipes(id)
    ) ''')

    # Sauvegarder les modifications et fermer la connexion à la base de données
    bdd.commit()
    bdd.close()

In [4]:
def setup_database(datapath):
    bdd = sqlite3.connect(datapath)
    cur = bdd.cursor()

    cur.execute("DROP TABLE IF EXISTS matchs")
    cur.execute("DROP TABLE IF EXISTS equipes")

    cur.execute(''' 
    CREATE TABLE IF NOT EXISTS equipes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nom VARCHAR(30),
        elo FLOAT,
        federation VARCHAR(30)
    ) ''')

    cur.execute(''' 
    CREATE TABLE IF NOT EXISTS matchs (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date DATE,
        domicile INTEGER,
        exterieur INTEGER,
        score_dom INTEGER,
        score_ext INTEGER,
        cote_dom FLOAT,
        cote_nul FLOAT,        
        cote_ext FLOAT,
        competition VARCHAR(30),
        FOREIGN KEY (domicile) REFERENCES equipes(id),
        FOREIGN KEY (exterieur) REFERENCES equipes(id)
    ) ''')

    bdd.commit()
    bdd.close()

In [5]:
def probabilite(x, y):
    somme = x + y
    prob_x = x / somme
    prob_y = y / somme
    prob_nul = abs(x - y)
    return prob_x, prob_y, prob_nul

In [6]:
def cotes(probabilite):
    if probabilite > 0:
        return 1 / probabilite
    else:
        return float('inf')

In [7]:
def calcul_elo(s1, s2, prob_x, prob_y, k=10):
    if s1 > s2:
        score_x = 1
        score_y = 0
    elif s1 == s2:
        score_x = 0.5
        score_y = 0.5
    else:
        score_x = 0
        score_y = 1

    delta_x = k * (score_x - prob_x)
    delta_y = k * (score_y - prob_y)

    return delta_x, delta_y

In [8]:
def creer_groupe(n, liste):
    lst = [[] for i in range(n)]
    for i in range(len(liste)):
        lst[i % n].append(liste[i])
    return lst

In [9]:
def generate_ligue(liste, nom):
    matchs = []
    bdd = sqlite3.connect(DATAPATH)
    cur = bdd.cursor()
    
    if len(liste) % 2 == 1: 
        liste.append(None)
        
    for j in range(len(liste) - 1):
        for i in range(len(liste) // 2):
            domicile = liste[i]
            exterieur = liste[len(liste) - 1 - i]
            if domicile is not None and exterieur is not None:
                matchs.append((domicile, exterieur))
                cur.execute('''INSERT INTO matchs (date, domicile, exterieur, cote_dom, cote_nul, cote_ext, score_dom, score_ext, competition) 
                       VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)''', 
                    ('2023-05-01', domicile.get_nom(), exterieur.get_nom(), None, None, None, None, None, nom))
        liste.insert(1, liste.pop())
    
    bdd.commit()
    bdd.close()
    
    return matchs

In [10]:
class Equipe:
    def __init__(self, nom, federation):
        self.nom = nom
        self.elo = 1000
        self.federation = federation
    
    def get_nom(self):
        return self.nom
    
    def set_elo(self, elo):
        self.elo = elo

    def get_elo(self):
        return self.elo
        

In [11]:
def insert_equipes(liste, federation):
    bdd = sqlite3.connect(DATAPATH)
    cur = bdd.cursor()
    liste_equipe = []
    
    for equipe in liste:
        cur.execute("INSERT INTO equipes (nom, elo, federation) VALUES (?, ?, ?)", (equipe, 1000, federation))
        liste_equipe.append(Equipe(equipe, federation))
        
    bdd.commit()
    bdd.close()
    return liste_equipe

In [12]:
def match(equipe):
    domicile = equipe[0]
    exterieur = equipe[1]
    score_domicile, score_exterieur = 0, 0
    prob_x, prob_y, prob_nul = probabilite(domicile.get_elo(), exterieur.get_elo())
    cote_x = cotes(prob_x)
    cote_y = cotes(prob_y)
    cote_nul = cotes(prob_nul)

    domicile_elo, exterieur_elo = calcul_elo(score_domicile, score_exterieur, prob_x, prob_y)
    elo_dom = domicile.get_elo() + domicile_elo
    elo_ext = exterieur.get_elo() + exterieur_elo
    domicile.set_elo(elo_dom)
    exterieur.set_elo(elo_ext)

In [13]:
setup_database(DATAPATH)

liste_nom_equipe = ['Paris-SG', 'Lens', 'Lille', 'Lyon']
liste_equipe = insert_equipes(liste_nom_equipe, 'FFF')
generate_ligue(liste_equipe, 'ligue 1')

[(<__main__.Equipe at 0x230bdcb78f0>, <__main__.Equipe at 0x230bdc97560>),
 (<__main__.Equipe at 0x230bdcb5490>, <__main__.Equipe at 0x230bdc95fa0>),
 (<__main__.Equipe at 0x230bdcb78f0>, <__main__.Equipe at 0x230bdc95fa0>),
 (<__main__.Equipe at 0x230bdc97560>, <__main__.Equipe at 0x230bdcb5490>),
 (<__main__.Equipe at 0x230bdcb78f0>, <__main__.Equipe at 0x230bdcb5490>),
 (<__main__.Equipe at 0x230bdc95fa0>, <__main__.Equipe at 0x230bdc97560>)]